In [56]:
import numpy as np
import random as rnd
from tqdm import tqdm

In [57]:
def forward(A, B, pi, sequence, eval=False):
    T = len(sequence)
    N = A.shape[0]
    alpha = np.zeros((N, T))
    alpha[:, 0] = pi * B[:, sequence[0]]
    constant = np.ones(T)
    if not eval:
        constant[0] = 1/np.sum(alpha[:, 0])
        alpha[:, 0] *= constant[0]
    for i in range(1, T):
        alpha[:, i] = (alpha[:, i-1] @ A) * B[:, sequence[i]]
        if not eval:
            constant[i] = 1/np.sum(alpha[:, i])
            alpha[:, i] *= constant[i]
    return alpha, constant

def backward(A, B, sequence, constant, eval=False):
    T = len(sequence)
    N = A.shape[0]
    beta = np.zeros((N, T))
    beta[:, -1] = 1
    if not eval:
        beta[:, -1] *= constant[-1]
    for i in range(T-1)[::-1]:
        beta[:, i] = A @ (B[:, sequence[i+1]] * beta[:, i+1])
        if not eval:
            beta[:, i] *= constant[i]
    return beta

In [58]:
def init_hmm_params(num_states, num_observations):
    
    # Init transition matrix with regards to the given map
    A = np.zeros((num_states, num_states))
    A[0, [0, 1, 5]] = 1
    A[1, [1, 0, 2]] = 1
    A[2, [2, 1, 3]] = 1
    A[3, [3, 2, 4, 8]] = 1
    A[4, [4, 3, 9]] = 1
    A[5, [5, 0, 6, 10]] = 1
    A[6, [6, 5, 7, 11, 12]] = 1
    A[7, [7, 6, 8, 12]] = 1
    A[8, [8, 3, 7, 9, 12, 13]] = 1
    A[9, [9, 4, 8, 14]] = 1
    A[10, [10, 5, 15]] = 1
    A[11, [11, 6, 12, 16]] = 1
    A[12, [12, 6, 7, 8, 11, 13, 16, 17, 18]] = 1
    A[13, [13, 8, 12, 14, 18]] = 1
    A[14, [14, 9, 13, 19]] = 1
    A[15, [15, 10, 20]] = 1
    A[16, [16, 11, 12, 17, 21]] = 1
    A[17, [17, 12, 16, 18, 22]] = 1
    A[18, [18, 12, 13, 17, 19, 23]] = 1
    A[19, [19, 14, 18, 24]] = 1
    A[20, [20, 15, 21]] = 1
    A[21, [21, 16, 20, 22]] = 1
    A[22, [22, 17, 21, 23]] = 1
    A[23, [23, 18, 22, 24]] = 1
    A[24, [24, 19, 23]] = 1
    assert (A - A.T).sum() == 0
    A = (A.T / A.sum(axis=1)).T
    
    # Init emission probabilities
    B = np.random.rand(num_states, num_observations)
    B = (B.T / B.sum(axis=1)).T

    # Init initial probabilities
    pi = np.random.rand(num_states)
    pi /= pi.sum()

    return A, B, pi

In [59]:
def init_transition_matrix_with_removed_edges(num_states):

    A = np.zeros((num_states, num_states))
    A[0, [0, 1, 5]] = 1
    A[1, [1, 0, 2]] = 1
    A[2, [2, 1, 3]] = 1
    A[3, [3, 2, 4, 8]] = 1
    A[4, [4, 3, 9]] = 1
    A[5, [5, 0, 6, 10]] = 1
    A[6, [6, 5, 7, 11]] = 1                                 # removed 6-12
    A[7, [7, 6, 8, 12]] = 1
    A[8, [8, 3, 7, 9, 13]] = 1                              # removed 8-12
    A[9, [9, 4, 8, 14]] = 1
    A[10, [10, 5, 15]] = 1
    A[11, [11, 6, 12, 16]] = 1
    A[12, [12, 7, 11, 16, 17, 18]] = 1                      # removed 12-8, 12-13, 12-6
    A[13, [13, 8, 12, 14, 18]] = 1                          # removed 13-12
    A[14, [14, 9, 13, 19]] = 1
    A[15, [15, 10, 20]] = 1
    A[16, [16, 11, 12, 21]] = 1                             # removed 16-17
    A[17, [17, 12, 18, 22]] = 1                             # removed 17-16
    A[18, [18, 12, 13, 17, 23]] = 1                         # removed 18-19
    A[19, [19, 14, 24]] = 1                                 # removed 19-18
    A[20, [20, 15, 21]] = 1
    A[21, [21, 16, 20, 22]] = 1
    A[22, [22, 17, 21, 23]] = 1
    A[23, [23, 18, 22, 24]] = 1
    A[24, [24, 19, 23]] = 1
    assert (A - A.T).sum() == 0
    A = (A.T / A.sum(axis=1)).T

    return A
    

# **Quesion 1: Learn HMM Parameters**

In [66]:
def baum_welch(sequences, iters=200):

    A, B, pi = init_hmm_params(25, 11)
    
    N = A.shape[0]
    O = B.shape[1]
    T = len(sequences[0])

    for i in range(iters):

        A_num = np.zeros((N, N))           # (N, N)
        A_denom = np.zeros((N, 1))         # (N, 1)
        B_num = np.zeros((N, O))           # (N, O)
        B_denom = np.zeros((N, 1))         # (N, 1)
        pi_num = np.zeros(N)          

        for seq_num in tqdm(range(sequences.shape[0]), bar_format='{l_bar}{bar:100}{r_bar}{bar:-10b}'):
            
            sequence = sequences[seq_num]
        
            alpha, constant = forward(A, B, pi, sequence)         # alpha: (N, T)
            beta = backward(A, B, sequence, constant)             # beta: (N, T)

            # xi: (N, N, T-1)
            # compute xi vectorized 
            alpha_t = alpha[:, :-1]
            beta_t = beta[:, 1:]
            B_t = B[:, sequence[1:]]
            numerator = (A[:, :, None] * alpha_t[:, None, :]) * (beta_t * B_t)
            denom = numerator.sum(axis=(0, 1))
            xi = numerator / (denom + 1e-7)

            gamma_temp = alpha * beta                                # gamma_temp: (N, T)
            gamma = gamma_temp / (gamma_temp.sum(axis=0) + 1e-7)     # gamma: (N, T)

            A_num += xi.sum(axis = 2)                                      # A_num: (N, N)
            A_denom += gamma[:, :-1].sum(axis=1).reshape(-1, 1)            # A_denom: (N, 1)

            for o in range(O):
                B_num[:, o] += np.sum(gamma[:, sequence == o], axis=1)
            B_denom += np.sum(gamma, axis=1).reshape(-1, 1)

            pi_num += gamma[:, 0]

        A_new = A_num / (A_denom + 1e-7)
        B_new = B_num / (B_denom + 1e-7)
        pi_new = pi_num / sequences.shape[1]

        print('A change: {}'.format(np.sum(np.abs(A_new - A))))
        print('B change: {}'.format(np.sum(np.abs(B_new - B))))
        print('pi change: {}'.format(np.sum(np.abs(pi_new - pi))))

        A = A_new
        B = B_new
        pi = pi_new


    return A, B, pi

In [67]:
train_data = np.load('train_data.npy')

A, B, pi = baum_welch(train_data, iters=100)
np.save('outputs/1_transition_matrix.npy', A)
np.save('outputs/2_emission_matrix.npy', B)
np.save('outputs/3_initial_dist.npy', pi)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 62.94it/s]


A change: 2.2550103586604924
B change: 12.398190422618763
pi change: 0.2347761903047734


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 65.98it/s]


A change: 1.1759100462348493
B change: 4.243546598369612
pi change: 0.1195047501268317


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 61.01it/s]


...

A change: 0.11700238194520873
B change: 0.13840470539653443
pi change: 0.004169702497883585


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 57.66it/s]


A change: 0.11495373972692627
B change: 0.1396925130968785
pi change: 0.003956935478581745


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 59.84it/s]

A change: 0.11274278613089578
B change: 0.14135961750496723
pi change: 0.0037903820547274775


# **Quesion 2: Compute sequence probabilities**

In [68]:
def forward_backward(sequence, A, B, pi):
    T = len(sequence)
    idx = np.random.randint(1, T)
    alpha, c = forward(A, B, pi, sequence, eval=True)
    beta = backward(A, B, sequence, c, eval=True)
    prob = alpha[:, idx] @ beta[:, idx]
    return prob

test_data = np.load('test_data.npy')
test_probs = np.zeros(test_data.shape[0])
for i, observation in enumerate(test_data):
    prob = forward_backward(observation, A, B, pi)
    test_probs[i] = prob
np.save('outputs/4_evaluation_problem.npy', test_probs)

# **Quesion 3: Most probable state sequence**

In [69]:
def viterbi(sequence, A, B, pi):
    T = len(sequence)
    N = A.shape[0]
    delta = np.zeros((N, T))
    backtrack = np.zeros((N, T), dtype=np.int32)
    delta[:, 0] = np.log(pi) + np.log(B[:, sequence[0]])
    backtrack[:, 0] = 0
    for i in range(1, T):
        temp = (np.log(A) + delta[:, i-1].reshape(-1, 1)) + np.log(B[:, sequence[i]])
        delta[:, i] = np.max(temp, axis=0)
        backtrack[:, i] = np.argmax(temp, axis=0)
    current_state = np.argmax(delta[:, -1])
    state_seq = [current_state]
    total_prob = np.exp(delta[current_state, -1])
    for t in range(1, T)[::-1]:
        prev_state = backtrack[current_state, t]
        state_seq.append(prev_state)
        current_state = prev_state
    return total_prob, state_seq[::-1]



In [70]:
A = np.load('outputs/1_transition_matrix.npy')
B = np.load('outputs/2_emission_matrix.npy')
pi = np.load('outputs/3_initial_dist.npy')
test_data = np.load('test_data.npy')

test_state_sequences = np.zeros_like(test_data)
for i, observation in enumerate(test_data):
    prob, state_seq = viterbi(observation, A, B, pi)
    test_state_sequences[i, :] = state_seq
np.save('outputs/5_best_paths.npy', test_state_sequences)

E:\Python Venv\mfi\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


# **Quesion 4**

In [71]:
P = np.load('outputs/1_transition_matrix.npy')
Q = init_transition_matrix_with_removed_edges(25)
new_P = np.zeros((25, 25))

a, b = np.linalg.eig(P.T)
c = b[:, a.argmax()]
target_steady = np.real(c / c.sum())
# print(target_steady)

current_state = np.random.randint(0, 25)

L = 1000000
state_probs = np.zeros((25, ))
state_probs[current_state] += 1

for _ in range(L):
    next_state = rnd.choices(range(0, 25), weights=Q[current_state, :])[0]
    aij = min(1, (target_steady[next_state] * Q[next_state, current_state])/(target_steady[current_state] * Q[current_state, next_state] + 1e-10))
    if rnd.uniform(0, 1) < aij:
        new_P[current_state, next_state] += 1
        current_state = next_state
    else:
        new_P[current_state, current_state] += 1
        current_state = current_state
    state_probs[current_state] += 1

state_probs = state_probs / L
new_P = new_P / new_P.sum(axis=1)[:, None]

new_steady = np.linalg.matrix_power(new_P, 2000)[0, :]
np.save('outputs/6_transition_after_edge_removal.npy', new_P)
print(np.abs(new_steady - target_steady))

[8.30689115e-04 1.69093041e-05 2.15146269e-04 2.71379171e-04
 2.30843830e-04 7.49435658e-04 8.51770621e-05 5.91516624e-05
 1.99656843e-04 6.70877435e-05 2.63421444e-04 1.11450880e-04
 6.97044198e-05 3.32440353e-04 3.40239926e-05 6.47014845e-04
 1.69547126e-04 5.22508701e-04 2.64834854e-04 2.16257265e-05
 7.95967952e-04 8.62717567e-05 7.56529870e-04 3.95687578e-04
 4.80744045e-05]
